In [ ]:
username = ""
source = "";

In [ ]:
import NBInclude: @nbinclude
@nbinclude("RecommendationsBase.ipynb");

In [ ]:
for task in ALL_TASKS
    get_twostage_recs(task)
end

In [ ]:
for task in ALL_TASKS
    get_onestage_recs(task)
end